# Wireless FiDEATHlity

## Objective

"Suicide is one of the leading causes of death among all American adults and rates are increasing in both men and women"(SSM Health).


## Research Questions

1. Which country has the most suicides ? Which country has the least suicides ?

2. What is the relationship between gender and the number of suicides?

3. Is there a relation about people with internet access, eating disorders and depression?

4. How is the relation between internet access and suicide rates by country?

### Import Libraries

In [24]:
import requests
import json
import pandas as pd

r = requests.get('http://apps.who.int/gho/athena/api/GHO/MH_12/.json?filter=Year:2019')
data = r.json()

### Convert JSON to Python list

In [25]:
list_of_data = []
for fact in data['fact']:
    for category in fact['Dim']:
        if category['category'] == 'COUNTRY':
            country_code = category['code']
        elif category['category'] == "SEX":
            sex = category['code']
    suicide_rate = fact['value']['numeric']
    list_of_data.append({'Country Code': country_code, 'Sex': sex, "Suicide Rate": suicide_rate})

country_codes = []
for country in list_of_data:
    country_codes.append(country['Country Code'])

country_codes = set(country_codes)

data_dictionary = []
for country in country_codes:
    data_dictionary.append({'Country Code': country, 'Male Suicide Rate':'', 'Female Suicide Rate':'', 'Combined Suicide Rate':''})

### Creating Python List with dictionary

In [26]:
for data in list_of_data:
    for country in data_dictionary:
        if data['Country Code'] == country['Country Code']:
            if data['Sex'] == 'MLE':
                country['Male Suicide Rate'] = data['Suicide Rate']
            elif data['Sex'] == "FMLE":
                country['Female Suicide Rate'] = data['Suicide Rate']
            elif data['Sex'] == 'BTSX':
                country['Combined Suicide Rate'] = data['Suicide Rate']

##### My first precious dataset <3

In [27]:
suicide_rates = pd.DataFrame.from_dict(data_dictionary,)
#suicide_rates.set_index(['Country Code'], inplace=True)
#suicide_rates.sort_index(inplace=True)
suicide_rates

,Country Code,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate
0,HRV,17.6925,5.08,11.0095
1,ATG,0.0000,0.60,0.3200
2,LAO,8.6300,3.45,5.9800
3,NOR,13.4010,6.29,9.9100
4,LUX,11.8095,5.43,8.6200
...,...,...,...,...
178,LBN,3.8700,1.67,2.7600
179,MKD,10.9785,3.53,7.1500
180,SLV,11.1394,2.08,6.1100
181,DOM,8.5300,1.85,5.1100


### Add Country Name column

In [28]:
#!pip install pycountry
import pandas as pd
import pycountry

countries = pd.DataFrame()
data = []
for country in pycountry.countries:
    try:
        current = {'Country Code': country.alpha_3,
                    'name': country.name,
                    'official_name': country.official_name}
    except Exception as ex:
        current = {'Country Code': country.alpha_3,
                    'name': country.name,
                    'official_name': None}        
    data.append(current)

countries = pd.DataFrame(data, columns = ['Country Code', 'name', 'official_name'])
countries = countries.drop('official_name', axis=1)

In [29]:
countries

,Country Code,name
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,AIA,Anguilla
4,ALA,Åland Islands
...,...,...
244,WSM,Samoa
245,YEM,Yemen
246,ZAF,South Africa
247,ZMB,Zambia


In [30]:
#suicide_rate = pd.concat([suicide_rates, country], axis=1, join='inner')

suicide_rate= pd.merge(suicide_rates, countries, on='Country Code')

In [31]:
suicide_rate

,Country Code,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,name
0,HRV,17.6925,5.08,11.0095,Croatia
1,ATG,0.0000,0.60,0.3200,Antigua and Barbuda
2,LAO,8.6300,3.45,5.9800,Lao People's Democratic Republic
3,NOR,13.4010,6.29,9.9100,Norway
4,LUX,11.8095,5.43,8.6200,Luxembourg
...,...,...,...,...,...
178,LBN,3.8700,1.67,2.7600,Lebanon
179,MKD,10.9785,3.53,7.1500,North Macedonia
180,SLV,11.1394,2.08,6.1100,El Salvador
181,DOM,8.5300,1.85,5.1100,Dominican Republic


### Check for Nans

In [32]:
suicide_rate.isna().sum()

Country Code             0
Male Suicide Rate        0
Female Suicide Rate      0
Combined Suicide Rate    0
name                     0
dtype: int64

In [33]:
suicide_rate.set_index(['Country Code'], inplace=True)
suicide_rate.sort_index(inplace=True)

### Importing percentage of population

In [34]:
r = requests.get('http://apps.who.int/gho/athena/api/GHO/AIR_4/?format=json')
data = r.json()

In [35]:
data_dictionary = []
for country in data['fact']:
    for category in country['Dim']:
        if category['category'] == "COUNTRY":
            countrycode = category['code']
    data_dictionary.append({"Country Code": countrycode, "% Living in Cities > 100k": country['value']['numeric']})

In [36]:
data_dictionary

[{'Country Code': 'AFG', '% Living in Cities > 100k': 2154.64204},
 {'Country Code': 'AFG', '% Living in Cities > 100k': 957.70975},
 {'Country Code': 'AFG', '% Living in Cities > 100k': 1196.93229},
 {'Country Code': 'AGO', '% Living in Cities > 100k': 2229.28417},
 {'Country Code': 'AGO', '% Living in Cities > 100k': 947.90736},
 {'Country Code': 'AGO', '% Living in Cities > 100k': 1281.37681},
 {'Country Code': 'ALB', '% Living in Cities > 100k': 3.449},
 {'Country Code': 'ALB', '% Living in Cities > 100k': 1.31357},
 {'Country Code': 'ALB', '% Living in Cities > 100k': 2.13543},
 {'Country Code': 'ARE', '% Living in Cities > 100k': 7.89728},
 {'Country Code': 'ARE', '% Living in Cities > 100k': 3.35645},
 {'Country Code': 'ARE', '% Living in Cities > 100k': 4.54083},
 {'Country Code': 'ARG', '% Living in Cities > 100k': 34.85399},
 {'Country Code': 'ARG', '% Living in Cities > 100k': 14.93785},
 {'Country Code': 'ARG', '% Living in Cities > 100k': 19.91614},
 {'Country Code': 'ARM'

In [40]:
percent_living_in_cities = pd.DataFrame(data_dictionary)
percent_living_in_cities.head()

,Country Code,% Living in Cities > 100k
0,AFG,2154.64204
1,AFG,957.70975
2,AFG,1196.93229
3,AGO,2229.28417
4,AGO,947.90736


##### OOPS SEEMS LIKE WE HAVE 3 VALUES OF EVERY COUNTRY... C1+C2=C3

I have 2 drop C1 and C2 which makes the sum of C3

In [41]:
percent_living_in_cities=percent_living_in_cities.drop_duplicates(subset=['Country Code'])

In [42]:
percent_living_in_cities

,Country Code,% Living in Cities > 100k
0,AFG,2154.64204
3,AGO,2229.28417
6,ALB,3.44900
9,ARE,7.89728
12,ARG,34.85399
...,...,...
534,WSM,0.35679
537,YEM,1802.18178
540,ZAF,1116.16673
543,ZMB,618.69992


In [61]:
suicide_rate= pd.merge(percent_living_in_cities, suicide_rate, on='Country Code')
suicide_rate.shape

(183, 7)

In [44]:
suicide_rate['% Living in Cities > 100k'].fillna(suicide_rate['% Living in Cities > 100k'].median(), inplace=True)
suicide_rate.isna().sum()

Country Code                 0
% Living in Cities > 100k    0
Male Suicide Rate            0
Female Suicide Rate          0
Combined Suicide Rate        0
name                         0
dtype: int64

### Internet access per country

In [92]:
internet = pd.read_csv('/Users/user/Downloads/share-of-individuals-using-the-internet.csv')
internet

,Entity,Code,Year,Individuals using the Internet (% of population)
0,Afghanistan,AFG,1990,0.000000
1,Afghanistan,AFG,1991,0.000000
2,Afghanistan,AFG,1992,0.000000
3,Afghanistan,AFG,1993,0.000000
4,Afghanistan,AFG,1994,0.000000
...,...,...,...,...
6528,Zimbabwe,ZWE,2016,23.119989
6529,Zimbabwe,ZWE,2017,24.400000
6530,Zimbabwe,ZWE,2018,25.000000
6531,Zimbabwe,ZWE,2019,25.100000


In [93]:
internet = internet.drop(internet[internet.Year != 2019].index)

In [94]:
internet = internet.rename(columns={"Code": "Country Code"})

In [95]:
internet_suicide= pd.merge(suicide_rate, internet, on='Country Code')
internet_suicide

,Country Code,% Living in Cities > 100k_x,% Living in Cities > 100k_y,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,name,Entity,Year,Individuals using the Internet (% of population)
0,AFG,2154.64204,2154.64204,6.1900,5.67,5.9600,Afghanistan,Afghanistan,2019,17.600000
1,AGO,2229.28417,2229.28417,21.7053,4.68,12.5546,Angola,Angola,2019,32.400002
2,ALB,3.44900,3.44900,5.2700,2.22,3.7200,Albania,Albania,2019,68.550392
3,ARE,7.89728,7.89728,6.2900,2.63,5.2400,United Arab Emirates,United Arab Emirates,2019,99.150002
4,ARG,34.85399,34.85399,13.4763,3.31,8.1400,Argentina,Argentina,2019,79.900002
...,...,...,...,...,...,...,...,...,...,...
148,UZB,317.69705,317.69705,11.8412,4.93,8.2800,Uzbekistan,Uzbekistan,2019,70.400002
149,VNM,619.54268,619.54268,10.5563,4.18,7.2200,Viet Nam,Vietnam,2019,68.661583
150,ZAF,1116.16673,1116.16673,37.8794,9.83,23.4856,South Africa,South Africa,2019,68.199997
151,ZMB,618.69992,618.69992,25.7417,5.28,14.4258,Zambia,Zambia,2019,16.799999


In [96]:
internet_suicide=internet_suicide.drop(columns={'% Living in Cities > 100k_x','Entity','Year'})
internet_suicide

,Country Code,% Living in Cities > 100k_y,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,name,Individuals using the Internet (% of population)
0,AFG,2154.64204,6.1900,5.67,5.9600,Afghanistan,17.600000
1,AGO,2229.28417,21.7053,4.68,12.5546,Angola,32.400002
2,ALB,3.44900,5.2700,2.22,3.7200,Albania,68.550392
3,ARE,7.89728,6.2900,2.63,5.2400,United Arab Emirates,99.150002
4,ARG,34.85399,13.4763,3.31,8.1400,Argentina,79.900002
...,...,...,...,...,...,...,...
148,UZB,317.69705,11.8412,4.93,8.2800,Uzbekistan,70.400002
149,VNM,619.54268,10.5563,4.18,7.2200,Viet Nam,68.661583
150,ZAF,1116.16673,37.8794,9.83,23.4856,South Africa,68.199997
151,ZMB,618.69992,25.7417,5.28,14.4258,Zambia,16.799999


In [97]:
internet_suicide.isna().sum()

Country Code                                        0
% Living in Cities > 100k_y                         0
Male Suicide Rate                                   0
Female Suicide Rate                                 0
Combined Suicide Rate                               0
name                                                0
Individuals using the Internet (% of population)    0
dtype: int64

### Eating Disorder

In [98]:
eatingdisorder = pd.read_csv('/Users/user/Downloads/share-with-an-eating-disorder.csv')
eatingdisorder

,Entity,Code,Year,Prevalence - Eating disorders - Sex: Both - Age: Age-standardized (Percent)
0,Afghanistan,AFG,1990,0.13
1,Afghanistan,AFG,1991,0.13
2,Afghanistan,AFG,1992,0.12
3,Afghanistan,AFG,1993,0.12
4,Afghanistan,AFG,1994,0.11
...,...,...,...,...
6835,Zimbabwe,ZWE,2015,0.10
6836,Zimbabwe,ZWE,2016,0.10
6837,Zimbabwe,ZWE,2017,0.10
6838,Zimbabwe,ZWE,2018,0.10


In [104]:
eatingdisorder = eatingdisorder.rename(columns={'Prevalence - Eating disorders - Sex: Both - Age: Age-standardized (Percent)':'Eating Disorder(%)','Code':'Country Code'})
eatingdisorder

,Entity,Country Code,Year,Eating Disorder(%)
29,Afghanistan,AFG,2019,0.12
59,African Region (WHO),NaN,2019,0.11
89,Albania,ALB,2019,0.15
119,Algeria,DZA,2019,0.22
149,American Samoa,ASM,2019,0.13
...,...,...,...,...
6719,World Bank Lower Middle Income,NaN,2019,0.13
6749,World Bank Upper Middle Income,NaN,2019,0.18
6779,Yemen,YEM,2019,0.13
6809,Zambia,ZMB,2019,0.12


In [105]:
eatingdisorder = eatingdisorder.drop(eatingdisorder[eatingdisorder.Year != 2019].index)
eatingdisorder

,Entity,Country Code,Year,Eating Disorder(%)
29,Afghanistan,AFG,2019,0.12
59,African Region (WHO),NaN,2019,0.11
89,Albania,ALB,2019,0.15
119,Algeria,DZA,2019,0.22
149,American Samoa,ASM,2019,0.13
...,...,...,...,...
6719,World Bank Lower Middle Income,NaN,2019,0.13
6749,World Bank Upper Middle Income,NaN,2019,0.18
6779,Yemen,YEM,2019,0.13
6809,Zambia,ZMB,2019,0.12


In [108]:
internet_suicide_eatingdisorder= pd.merge(internet_suicide, eatingdisorder, on='Country Code')
internet_suicide_eatingdisorder

,Country Code,% Living in Cities > 100k_y,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,name,Individuals using the Internet (% of population),Entity,Year,Eating Disorder(%)
0,AFG,2154.64204,6.1900,5.67,5.9600,Afghanistan,17.600000,Afghanistan,2019,0.12
1,AGO,2229.28417,21.7053,4.68,12.5546,Angola,32.400002,Angola,2019,0.14
2,ALB,3.44900,5.2700,2.22,3.7200,Albania,68.550392,Albania,2019,0.15
3,ARE,7.89728,6.2900,2.63,5.2400,United Arab Emirates,99.150002,United Arab Emirates,2019,0.32
4,ARG,34.85399,13.4763,3.31,8.1400,Argentina,79.900002,Argentina,2019,0.36
...,...,...,...,...,...,...,...,...,...,...
148,UZB,317.69705,11.8412,4.93,8.2800,Uzbekistan,70.400002,Uzbekistan,2019,0.12
149,VNM,619.54268,10.5563,4.18,7.2200,Viet Nam,68.661583,Vietnam,2019,0.10
150,ZAF,1116.16673,37.8794,9.83,23.4856,South Africa,68.199997,South Africa,2019,0.17
151,ZMB,618.69992,25.7417,5.28,14.4258,Zambia,16.799999,Zambia,2019,0.12


In [109]:
internet_suicide_eatingdisorder.isna().sum()

Country Code                                        0
% Living in Cities > 100k_y                         0
Male Suicide Rate                                   0
Female Suicide Rate                                 0
Combined Suicide Rate                               0
name                                                0
Individuals using the Internet (% of population)    0
Entity                                              0
Year                                                0
Eating Disorder(%)                                  0
dtype: int64

In [110]:
internet_suicide_eatingdisorder = internet_suicide_eatingdisorder.drop(columns={'name','Year'})
internet_suicide_eatingdisorder

,Country Code,% Living in Cities > 100k_y,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,Individuals using the Internet (% of population),Entity,Eating Disorder(%)
0,AFG,2154.64204,6.1900,5.67,5.9600,17.600000,Afghanistan,0.12
1,AGO,2229.28417,21.7053,4.68,12.5546,32.400002,Angola,0.14
2,ALB,3.44900,5.2700,2.22,3.7200,68.550392,Albania,0.15
3,ARE,7.89728,6.2900,2.63,5.2400,99.150002,United Arab Emirates,0.32
4,ARG,34.85399,13.4763,3.31,8.1400,79.900002,Argentina,0.36
...,...,...,...,...,...,...,...,...
148,UZB,317.69705,11.8412,4.93,8.2800,70.400002,Uzbekistan,0.12
149,VNM,619.54268,10.5563,4.18,7.2200,68.661583,Vietnam,0.10
150,ZAF,1116.16673,37.8794,9.83,23.4856,68.199997,South Africa,0.17
151,ZMB,618.69992,25.7417,5.28,14.4258,16.799999,Zambia,0.12


### Depression Rate by country

In [112]:
depression = pd.read_csv('/Users/user/Downloads/share-with-depression.csv')
depression

,Entity,Code,Year,Prevalence - Depressive disorders - Sex: Both - Age: Age-standardized (Percent)
0,Afghanistan,AFG,1990,5.13
1,Afghanistan,AFG,1991,5.12
2,Afghanistan,AFG,1992,5.11
3,Afghanistan,AFG,1993,5.10
4,Afghanistan,AFG,1994,5.10
...,...,...,...,...
6747,Zimbabwe,ZWE,2015,3.55
6748,Zimbabwe,ZWE,2016,3.56
6749,Zimbabwe,ZWE,2017,3.56
6750,Zimbabwe,ZWE,2018,3.56


In [113]:
depression = depression.rename(columns={'Prevalence - Depressive disorders - Sex: Both - Age: Age-standardized (Percent)':'Depressive Disorder (%)','Code':'Country Code'})
depression

,Entity,Country Code,Year,Depressive Disorder (%)
0,Afghanistan,AFG,1990,5.13
1,Afghanistan,AFG,1991,5.12
2,Afghanistan,AFG,1992,5.11
3,Afghanistan,AFG,1993,5.10
4,Afghanistan,AFG,1994,5.10
...,...,...,...,...
6747,Zimbabwe,ZWE,2015,3.55
6748,Zimbabwe,ZWE,2016,3.56
6749,Zimbabwe,ZWE,2017,3.56
6750,Zimbabwe,ZWE,2018,3.56


In [114]:
depression = depression.drop(depression[depression.Year != 2019].index)
depression

,Entity,Country Code,Year,Depressive Disorder (%)
29,Afghanistan,AFG,2019,5.13
59,African Region (WHO),NaN,2019,4.59
89,Albania,ALB,2019,2.63
119,Algeria,DZA,2019,4.37
149,American Samoa,ASM,2019,2.80
...,...,...,...,...
6631,World Bank Lower Middle Income,NaN,2019,3.72
6661,World Bank Upper Middle Income,NaN,2019,3.32
6691,Yemen,YEM,2019,5.04
6721,Zambia,ZMB,2019,4.30


In [115]:
depression = depression.drop(columns={'Entity','Year'})
depression

,Country Code,Depressive Disorder (%)
29,AFG,5.13
59,NaN,4.59
89,ALB,2.63
119,DZA,4.37
149,ASM,2.80
...,...,...
6631,NaN,3.72
6661,NaN,3.32
6691,YEM,5.04
6721,ZMB,4.30


### Final Data

In [116]:
data= pd.merge(internet_suicide_eatingdisorder, depression, on='Country Code')
data

,Country Code,% Living in Cities > 100k_y,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,Individuals using the Internet (% of population),Entity,Eating Disorder(%),Depressive Disorder (%)
0,AFG,2154.64204,6.1900,5.67,5.9600,17.600000,Afghanistan,0.12,5.13
1,AGO,2229.28417,21.7053,4.68,12.5546,32.400002,Angola,0.14,5.86
2,ALB,3.44900,5.2700,2.22,3.7200,68.550392,Albania,0.15,2.63
3,ARE,7.89728,6.2900,2.63,5.2400,99.150002,United Arab Emirates,0.32,3.76
4,ARG,34.85399,13.4763,3.31,8.1400,79.900002,Argentina,0.36,2.59
...,...,...,...,...,...,...,...,...,...
146,UZB,317.69705,11.8412,4.93,8.2800,70.400002,Uzbekistan,0.12,3.34
147,VNM,619.54268,10.5563,4.18,7.2200,68.661583,Vietnam,0.10,2.58
148,ZAF,1116.16673,37.8794,9.83,23.4856,68.199997,South Africa,0.17,4.45
149,ZMB,618.69992,25.7417,5.28,14.4258,16.799999,Zambia,0.12,4.30


##### I decide to remove the column % Living in Cities > 100k_y and just use the total number of population

In [125]:
population = pd.read_csv('/Users/user/Downloads/population-since-1800.csv')
population = population.drop(population[population.Year != 2019].index)
population

,Entity,Code,Year,Population (historical estimates)
219,Afghanistan,AFG,2019,37769496
441,Africa,NaN,2019,1327701380
663,Albania,ALB,2019,2873883
885,Algeria,DZA,2019,42705372
972,American Samoa,ASM,2019,47341
...,...,...,...,...
48511,Western Sahara,ESH,2019,544883
48733,World,OWID_WRL,2019,7764951040
48955,Yemen,YEM,2019,31546694
49177,Zambia,ZMB,2019,18380478


In [126]:
population = population.rename(columns={'Population (historical estimates)':'Population','Code':'Country Code'})
population

,Entity,Country Code,Year,Population
219,Afghanistan,AFG,2019,37769496
441,Africa,NaN,2019,1327701380
663,Albania,ALB,2019,2873883
885,Algeria,DZA,2019,42705372
972,American Samoa,ASM,2019,47341
...,...,...,...,...
48511,Western Sahara,ESH,2019,544883
48733,World,OWID_WRL,2019,7764951040
48955,Yemen,YEM,2019,31546694
49177,Zambia,ZMB,2019,18380478


In [127]:
population = population.drop(columns={'Entity','Year'})
population

,Country Code,Population
219,AFG,37769496
441,NaN,1327701380
663,ALB,2873883
885,DZA,42705372
972,ASM,47341
...,...,...
48511,ESH,544883
48733,OWID_WRL,7764951040
48955,YEM,31546694
49177,ZMB,18380478


In [128]:
data2= pd.merge(data, population, on='Country Code')
data2

,Country Code,% Living in Cities > 100k_y,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,Individuals using the Internet (% of population),Entity,Eating Disorder(%),Depressive Disorder (%),Population
0,AFG,2154.64204,6.1900,5.67,5.9600,17.600000,Afghanistan,0.12,5.13,37769496
1,AGO,2229.28417,21.7053,4.68,12.5546,32.400002,Angola,0.14,5.86,32353592
2,ALB,3.44900,5.2700,2.22,3.7200,68.550392,Albania,0.15,2.63,2873883
3,ARE,7.89728,6.2900,2.63,5.2400,99.150002,United Arab Emirates,0.32,3.76,9211660
4,ARG,34.85399,13.4763,3.31,8.1400,79.900002,Argentina,0.36,2.59,44745516
...,...,...,...,...,...,...,...,...,...,...
146,UZB,317.69705,11.8412,4.93,8.2800,70.400002,Uzbekistan,0.12,3.34,32976946
147,VNM,619.54268,10.5563,4.18,7.2200,68.661583,Vietnam,0.10,2.58,95776712
148,ZAF,1116.16673,37.8794,9.83,23.4856,68.199997,South Africa,0.17,4.45,58087060
149,ZMB,618.69992,25.7417,5.28,14.4258,16.799999,Zambia,0.12,4.30,18380478


In [132]:
data_final = data2.drop(columns={'% Living in Cities > 100k_y'})
data_final

,Country Code,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,Individuals using the Internet (% of population),Entity,Eating Disorder(%),Depressive Disorder (%),Population
0,AFG,6.1900,5.67,5.9600,17.600000,Afghanistan,0.12,5.13,37769496
1,AGO,21.7053,4.68,12.5546,32.400002,Angola,0.14,5.86,32353592
2,ALB,5.2700,2.22,3.7200,68.550392,Albania,0.15,2.63,2873883
3,ARE,6.2900,2.63,5.2400,99.150002,United Arab Emirates,0.32,3.76,9211660
4,ARG,13.4763,3.31,8.1400,79.900002,Argentina,0.36,2.59,44745516
...,...,...,...,...,...,...,...,...,...
146,UZB,11.8412,4.93,8.2800,70.400002,Uzbekistan,0.12,3.34,32976946
147,VNM,10.5563,4.18,7.2200,68.661583,Vietnam,0.10,2.58,95776712
148,ZAF,37.8794,9.83,23.4856,68.199997,South Africa,0.17,4.45,58087060
149,ZMB,25.7417,5.28,14.4258,16.799999,Zambia,0.12,4.30,18380478


### Clean the Final Data

In [140]:
data_final = data_final.rename(columns={'Individuals using the Internet (% of population)':'Internet Access (%)'})

In [143]:
data_final = data_final[['Country Code','Entity','Population','Internet Access (%)','Eating Disorder(%)','Depressive Disorder (%)','Combined Suicide Rate', 'Male Suicide Rate', 'Female Suicide Rate']]
data_final = data_final.set_index('Country Code')

In [144]:
data_final

,Entity,Population,Internet Access (%),Eating Disorder(%),Depressive Disorder (%),Combined Suicide Rate,Male Suicide Rate,Female Suicide Rate
Country Code,,,,,,,,
AFG,Afghanistan,37769496,17.600000,0.12,5.13,5.9600,6.1900,5.67
AGO,Angola,32353592,32.400002,0.14,5.86,12.5546,21.7053,4.68
ALB,Albania,2873883,68.550392,0.15,2.63,3.7200,5.2700,2.22
ARE,United Arab Emirates,9211660,99.150002,0.32,3.76,5.2400,6.2900,2.63
ARG,Argentina,44745516,79.900002,0.36,2.59,8.1400,13.4763,3.31
...,...,...,...,...,...,...,...,...
UZB,Uzbekistan,32976946,70.400002,0.12,3.34,8.2800,11.8412,4.93
VNM,Vietnam,95776712,68.661583,0.10,2.58,7.2200,10.5563,4.18
ZAF,South Africa,58087060,68.199997,0.17,4.45,23.4856,37.8794,9.83


In [145]:
data_final.to_csv('final_data.csv')